Obtain CNN URL's

In [1]:
# Import necessary libraries
import time
import requests
import urllib
import lxml.html
from bs4 import BeautifulSoup
import newspaper
import trafilatura
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [12]:
# CNN


def obtain_page_urls(from_art="0", page="1"):
    url = "https://www.cnn.com/search?q=politics&from={}&size=10&page={}&sort=newest&types=all&section=politics".format(
        from_art, page
    )
    resp = requests.get(url)
    root = lxml.html.fromstring(resp.text)
    div_list = root.cssselect("div.search__results-list")
    print("Div_list", div_list)
    article_links = div_list[0].cssselect("a")
    article_list = []
    a_elements = root.cssselect("a.container__link")
    print(a_elements)
    # for link in article_links:
    #    article_list.append(link.get("href"))


obtain_page_urls()

Div_list [<Element div at 0x7f84e9239180>]
[]


In [ ]:
def obtain_cnn_urls(max_pages_to_crawl = 50):
    cnn_links = []
    url = "https://www.cnn.com/search?q=politics&from=0&size=10&page=1&sort=newest&types=all&section=politics"
    resp = requests.get(url)
    root = lxml.html.fromstring(resp.text)
    ##

    max_pages_text = root.cssselect("div.search__results-count")[0]
    ##Write lines to obtain number of pages 
    max_pages = 
    if max_pages < max_pages_to_crawl:
        pages_to_crawl = max_pages
    else:
        pages_to_crawl = max_pages_to_crawl

    #Crawl and retrieve Urls using helper function
    from_art = 0
    page = 1

    for page in range(0,max_pages):
        cnn_links.extend(obtain_page_urls(from_art,page))
        from_art += 10
        page += 1

    cnn_links = set(cnn_links)
    return cnn_links

Scrape with Selenium

In [37]:
def obtain_page_urls(driver):
    page_list = []
    # url = "https://www.cnn.com/search?q=politics&from={}&size=10&page={}&sort=newest&types=all&section=politics".format(from_art,page)
    # # driver = webdriver.Chrome()
    # driver.get(url)
    links = driver.find_elements(By.CSS_SELECTOR, "a.container__link")

    for link in links:
        page_list.append(link.get_attribute("href"))
    return page_list


# driver = webdriver.Chrome()
# obtain_page_urls()

In [35]:
def move_next_page(driver):
    time.sleep(4)
    next_button = driver.find_elements(By.CSS_SELECTOR, "div.pagination-arrow-right")

    if len(next_button) > 0:
        exists = True
    else:
        exists = False

    if exists:
        next_button[0].click()
        return exists

    return exists


# move_next_page()

In [47]:
def obtain_cnn_urls(max_pages_to_crawl=10):
    links_list = []
    driver = webdriver.Chrome()
    driver.get(
        "https://www.cnn.com/search?q=politics&from=0&size=10&page=1&sort=newest&types=all&section=politics"
    )
    time.sleep(20)
    for i in range(max_pages_to_crawl):
        driver.implicitly_wait(5)
        page = i + 1
        print("Start page:", page)
        print(str(driver.current_url))
        # Obtain links
        links_list.extend(obtain_page_urls(driver))
        driver.implicitly_wait(5)
        # Move to next page or break crawler
        next_page = move_next_page(driver)
        if next_page == False:
            break

    links_set = set(links_list)

    return links_set

In [ ]:
obtain_cnn_urls(10)

In [57]:
url = "https://search.api.cnn.com/content?q=politics&size=10&from=40&page=5&sort=newest&sections=politics"
resp = requests.get(url)
root = resp.json()

root

{'message': 'success',
 'result': [{'_id': 'h_effac3bff28554b998124fd44fadada5',
   'type': 'video',
   'sourceId': 'video_7415005E-BC28-6D11-023D-3AE5964F8515',
   'url': 'https://www.cnn.com/videos/politics/2023/07/09/biden-nato-bid-ukraine-zelensky-michael-mccaul-sotu-vpx.cnn',
   'path': '/videos/politics/2023/07/09/biden-nato-bid-ukraine-zelensky-michael-mccaul-sotu-vpx.cnn',
   'source': 'cnn',
   'firstPublishDate': '2023-07-09T14:41:17Z',
   'lastPublishDate': '2023-07-10T11:05:09Z',
   'lastModifiedDate': '2023-07-10T11:05:09Z',
   'headline': 'GOP Rep. McCaul on how Ukraine can gain NATO membership',
   'section': 'politics',
   'mappedSection': 'POLITICS',
   'body': "Rep. Michael McCaul (R-TX) tells CNN's Jake Tapper he thinks Ukraine should be admitted to NATO after the war, but they would need to meet certain qualifications.",
   'thumbnail': 'https://cdn.cnn.com/cnnnext/dam/assets/221115203453-nato-file-2002-story-body.jpeg',
   'trt': 112.84617951284618,
   'contributor

Selenium is too slow and it was possible to find the API request from CNN

In [2]:
def obtain_page_urls(from_art="0", page="1"):
    page_links = set()
    url = "https://search.api.cnn.com/content?q=politics&size=10&from={}&page={}&sort=newest&sections=politics".format(
        from_art, page
    )
    resp = requests.get(url)
    resp_json = resp.json()
    for article in resp_json["result"]:
        page_links.add(article["url"])
        year = re.search(r"\d{4}", article["lastPublishDate"]).group()

    return page_links, year

In [3]:
def obtain_cnn_urls(min_year=2016):
    cnn_links = set()
    # Crawl and retrieve Urls using helper function
    from_art = 0
    page = 1
    year = 9999

    page_set_len = 0

    while int(year) >= min_year:
        print("Obtaining results for page", page)
        page_urls, year = obtain_page_urls(from_art, page)
        cnn_links.update(page_urls)
        from_art += 10
        page += 1
        len_set = len(cnn_links)
        if len_set == page_set_len:
            break
        else:
            page_set_len = len_set

    return cnn_links

In [4]:
test_cnn_urls = obtain_cnn_urls(2022)
len(test_cnn_urls)
test_cnn_urls

Obtaining results for page 1
Obtaining results for page 2
Obtaining results for page 3
Obtaining results for page 4
Obtaining results for page 5
Obtaining results for page 6
Obtaining results for page 7
Obtaining results for page 8
Obtaining results for page 9
Obtaining results for page 10
Obtaining results for page 11
Obtaining results for page 12
Obtaining results for page 13
Obtaining results for page 14
Obtaining results for page 15
Obtaining results for page 16
Obtaining results for page 17
Obtaining results for page 18
Obtaining results for page 19
Obtaining results for page 20
Obtaining results for page 21
Obtaining results for page 22
Obtaining results for page 23
Obtaining results for page 24
Obtaining results for page 25
Obtaining results for page 26
Obtaining results for page 27
Obtaining results for page 28
Obtaining results for page 29
Obtaining results for page 30
Obtaining results for page 31
Obtaining results for page 32
Obtaining results for page 33
Obtaining results f

{'https://www.cnn.com/2023/04/14/politics/mike-pompeo/index.html',
 'https://www.cnn.com/2013/08/20/us/bob-graham-fast-facts/index.html',
 'https://www.cnn.com/2023/01/27/politics/americas-week-of-tragedy-analysis/index.html',
 'https://www.cnn.com/2023/06/15/politics/trump-documents-case-cannon-security-clearances/index.html',
 'https://www.cnn.com/2022/11/17/politics/pence-cnn-town-hall-january-6/index.html',
 'https://www.cnn.com/2022/08/30/politics/trump-allies-biden-midterm-election-heat-analysis/index.html',
 'https://www.cnn.com/2023/02/22/politics/garret-miller-sentenced-alexandria-ocasio-cortez-threats/index.html',
 'https://www.cnn.com/2022/09/06/politics/south-carolina-near-total-abortion-ban-exceptions/index.html',
 'https://www.cnn.com/2023/06/07/politics/trump-documents-florida-grand-jury-explainer/index.html',
 'https://www.cnn.com/2023/06/27/politics/north-carolina-abortion-law-changes-lawsuit/index.html',
 'https://www.cnn.com/2022/09/09/politics/barack-obama-malevolen

({'https://www.cnn.com/2022/06/21/politics/alabama-virginia-georgia-dc-primary-election/index.html',
  'https://www.cnn.com/2022/06/21/politics/ivanka-trump-election-challenge-documentary/index.html',
  'https://www.cnn.com/2022/06/21/politics/mayra-flores-first-mexican-born-woman-congress/index.html',
  'https://www.cnn.com/2022/06/21/politics/ron-johnson-fake-elector-scheme-trump/index.html',
  'https://www.cnn.com/2022/06/22/politics/detainee-family-frustrations-biden/index.html',
  'https://www.cnn.com/2022/06/22/politics/georgia-election-workers-american-democracy-trump/index.html',
  'https://www.cnn.com/2022/06/22/politics/john-roberts-maine-church-state/index.html',
  'https://www.cnn.com/politics/live-news/january-6-hearings-june-21/index.html',
  'https://www.cnn.com/videos/politics/2022/06/22/ivanka-trump-2020-election-documentary-haberman-ac360-vpx.cnn',
  'https://www.cnn.com/videos/politics/2022/06/22/ron-johnson-pence-electors-manu-raju-2022-january-6-hearings-ac360-vpx.